In [1]:
import model
import numpy as np
import tensorflow as tf
import image_preprocessing
import matplotlib.pyplot as plt

In [3]:
import os
def print_directory_contents(directory):
    try:
        # List all files and directories in the specified directory
        entries = os.listdir(directory)
        print(f"Contents of '{directory}':")
        for entry in entries:
            print(entry)
    except FileNotFoundError:
        print(f"Directory '{directory}' not found.")
    except PermissionError:
        print(f"Permission denied to access '{directory}'.")

print_directory_contents("preprocessed_dataset/valid")

Contents of 'preprocessed_dataset/valid':
Apple___Apple_scab
Apple___Black_rot
Apple___Cedar_apple_rust
Apple___healthy
Blueberry___healthy
Cherry_(including_sour)___healthy
Cherry_(including_sour)___Powdery_mildew
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Corn_(maize)___Common_rust_
Corn_(maize)___healthy
Corn_(maize)___Northern_Leaf_Blight
Grape___Black_rot
Grape___Esca_(Black_Measles)
Grape___healthy
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
Orange___Haunglongbing_(Citrus_greening)
Peach___Bacterial_spot
Peach___healthy
Pepper,_bell___Bacterial_spot
Pepper,_bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Raspberry___healthy
Soybean___healthy
Squash___Powdery_mildew
Strawberry___healthy
Strawberry___Leaf_scorch
Tomato___Bacterial_spot
Tomato___Early_blight
Tomato___healthy
Tomato___Late_blight
Tomato___Leaf_Mold
Tomato___Septoria_leaf_spot
Tomato___Spider_mites Two-spotted_spider_mite
Tomato___Target_Spot
Tomato___Tomato_mosaic_virus
Tomato___Toma

In [5]:
import importlib
importlib.reload(image_preprocessing)

num_samples = 3200
input_shape = (128, 128, 3)
batch_size = 32

# Use the same label encoder for validation and test datasets
X_val, y_val, label_encoder = image_preprocessing.create_numpy_dataset(
    "preprocessed_dataset/valid"
)

# Convert the unint 8s to float for training and renormalize.
X_val = X_val.astype('float32') / 255.0


print(X_val.shape)
print(y_val.shape)

(17572, 128, 128, 3)
(17572,)


In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

num_classes = 38
y_val = to_categorical(y_val, num_classes=num_classes)

In [8]:
y_val_flat = np.argmax(y_val, axis=1) if len(y_val.shape) > 1 else y_val
print("Validation set classes:", set(y_val_flat))
print("Encoded classes:", label_encoder.classes_)

Validation set classes: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37}
Encoded classes: ['Apple___Apple_scab' 'Apple___Black_rot' 'Apple___Cedar_apple_rust'
 'Apple___healthy' 'Blueberry___healthy'
 'Cherry_(including_sour)___Powdery_mildew'
 'Cherry_(including_sour)___healthy'
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot'
 'Corn_(maize)___Common_rust_' 'Corn_(maize)___Northern_Leaf_Blight'
 'Corn_(maize)___healthy' 'Grape___Black_rot'
 'Grape___Esca_(Black_Measles)'
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)' 'Grape___healthy'
 'Orange___Haunglongbing_(Citrus_greening)' 'Peach___Bacterial_spot'
 'Peach___healthy' 'Pepper,_bell___Bacterial_spot'
 'Pepper,_bell___healthy' 'Potato___Early_blight' 'Potato___Late_blight'
 'Potato___healthy' 'Raspberry___healthy' 'Soybean___healthy'
 'Squash___Powdery_mildew' 'Strawberry___Leaf_scorch'
 'Strawberry___healthy' 'Tomato___Bacterial_spot' '

In [9]:
import pandas as pd

# Define the models to load
model_paths = [
    "plant_CNN_2Layers/plant_cnn_model.h5",
    "plant_resnet_3Layers/plant_resnet3_model.h5",
    "plant_resnet_10Layers/plant_resnet10_model.h5"
]

# Initialize a DataFrame to store results
results = []

# Loop through each model path
for model_path in model_paths:
    # Load the model
    plant_model = tf.keras.models.load_model(model_path)
    
    # Summarize the model to count the trainable parameters
    trainable_params = plant_model.count_params()
    
    # Evaluate the model on validation data
    val_loss, val_accuracy = plant_model.evaluate(X_val, y_val, batch_size=batch_size, verbose=1)
    
    # Append results to the DataFrame
    results.append({
        "Model": model_path.split('/')[-1],  # Get just the model name
        "Validation Loss": val_loss,
        "Validation Accuracy": val_accuracy,
        "Trainable Parameters": trainable_params
    })

# Create a pandas DataFrame from the results
results_df = pd.DataFrame(results)

# Display the DataFrame
print(results_df)

550/550 [==============================] - 13s 22ms/step - loss: 0.1997 - accuracy: 0.9397
                     Model  Validation Loss  Validation Accuracy  \
0       plant_cnn_model.h5         0.463933             0.856818   
1   plant_resnet3_model.h5         0.156372             0.950603   
2  plant_resnet10_model.h5         0.199659             0.939734   

   Trainable Parameters  
0                156070  
1                912934  
2               2986278  


In [10]:
from sklearn.metrics import classification_report
import numpy as np

# Placeholder to store classification reports
classification_reports = {}

for model_path in model_paths:
    # Load the model
    plant_model = tf.keras.models.load_model(model_path)
    
    # Predict on the validation data
    y_pred_probs = plant_model.predict(X_val, batch_size=batch_size, verbose=1)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_val, axis=1)
    
    # Map the indices to class names
    class_names = label_encoder.classes_  # Assuming label_encoder has a `classes_` attribute
    
    # Generate the classification report
    report = classification_report(
        y_true,
        y_pred,
        target_names=class_names,
        output_dict=True  # To save it as a structured dict for further processing if needed
    )
    
    # Store the report for the model
    classification_reports[model_path.split('/')[-1]] = report

    # Print the report
    print(f"Classification Report for {model_path.split('/')[-1]}")
    print(classification_report(y_true, y_pred, target_names=class_names))

# Optional: Convert all reports into a pandas DataFrame for better visualization
all_reports_df = pd.concat(
    {model: pd.DataFrame(report).transpose() for model, report in classification_reports.items()},
    axis=0
)
print(all_reports_df)

550/550 [==============================] - 3s 6ms/step
Classification Report for plant_cnn_model.h5
                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.95      0.72      0.82       504
                                 Apple___Black_rot       0.85      0.94      0.89       497
                          Apple___Cedar_apple_rust       0.93      0.75      0.83       440
                                   Apple___healthy       0.84      0.72      0.78       502
                               Blueberry___healthy       0.96      0.85      0.90       454
          Cherry_(including_sour)___Powdery_mildew       0.88      0.87      0.87       421
                 Cherry_(including_sour)___healthy       0.85      0.98      0.91       456
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.83      0.87      0.85       410
                       Corn_(maize)___Common_rust_       0.99      0.98

In [12]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)


In [13]:
# Export results to Excel
output_excel_file = "model_evaluation_results.xlsx"

# Use ExcelWriter to write multiple DataFrames to the same spreadsheet
with pd.ExcelWriter(output_excel_file, engine='openpyxl') as writer:
    # Write results_df to a sheet
    results_df.to_excel(writer, sheet_name="Model Results", index=False)
    
    # Write each classification report to separate sheets
    for model, report in classification_reports.items():
        # Convert the classification report dictionary to a DataFrame
        report_df = pd.DataFrame(report).transpose()
        # Write the DataFrame to a sheet named after the model
        report_df.to_excel(writer, sheet_name=model[:30])  # Sheet names are limited to 31 characters

print(f"Results and classification reports saved to {output_excel_file}")

Results and classification reports saved to model_evaluation_results.xlsx
